# Fine-Tuning de un LLM

Para trabajar el código siguiente, hicimos el uso exclusivo del siguiente libro:

- D. Voigt Godoy. (2025). A Hands-On Guide to Fine-Tuning Large Language Models with PyTorch and Hugging Face. Versión 1.0.1.

En dicho libro, Voigt explica, ejemplifica y realiza ejercicios de fine-tuning para distintos LLMs. En nuestro caso, el código siguiente se encuentra basado fuertemente en esta bibliografía, por lo que recomendamos que se pueda visitar el Repo de Github de dicho libro (https://github.com/dvgodoy/FineTuningLLMs).

También es necesario informar que con este archivo se llevó a cabo el fine-tuning del LLM `Phi-3` de Microsoft, disponible en los siguientes links:

- https://huggingface.co/microsoft/Phi-3-mini-4k-instruct


Por último, este script se corrió en Google Colab, ya que las GPUs disponibles cuentan con al menos 10GB de VRAM. Esto aseguró que nuestro entrenamiento se redujera exponencialmente, ya que  al tratar de hacer el fine tuning sobre una tarjeta  Geforce RTX 2070 Super de 8GB, encontramos  que el tiempo estimado de fine-tuning era de 3 días haciendo uso de solamente 12,500 muestras. En Google Colab este tiempo se redujo a 3 horas.

## Explicación del Fine-Tuning

Primero revisamos la tarjeta utilizada por Google Colab, esto para efectos de reproducibilidad.

In [ ]:
!nvidia-smi

Thu Oct  9 19:29:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Luego, instalamos las librearías necesarias para que Google Colab pueda llevar a cabo el fine-tuning.

Estas versiones de las librerías aseguran que existan compatibilidad entre todas ellas.

In [ ]:
#!pip install datasets bitsandbytes trl
!pip install transformers==4.55.2 peft==0.17.0 accelerate==1.10.0 trl==0.21.0 bitsandbytes==0.47.0 datasets==4.0.0 huggingface-hub==0.34.4 safetensors==0.6.2 pandas==2.2.2 matplotlib==3.10.0 numpy==2.0.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 143.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 117.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.35.3
    Uninstalling huggingface-hub-0.35.3:
      Successfully uninstalled huggingface-hub-0.35.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22

Importamos las librerías hacia nuestra máquina virtual (esto puede llevar un tiempo).

In [ ]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer


### Cuantización (Quantization)

La idea de cuantizar un LLM es que ésta no abarque mucha memoria dentro de la RAM de un GPU. La idea en específico es la siguiente:

- Se reemplazan los pesos o weights del modelo original con valores aproximados que se pueden representar en menos bits (por ejemplo, si un modelo usa pesos en 32 bits, podemos buscar representarlos en 4 bits para reducir el tamaño en un factor de 8 veces).
- El proceso de cuantización genera que el modelo "pierda" cierta exactitud o que el desempeño sea peor respecto a la carga completa del mismo.
- La cuantización se centra en las capas lineales dentro de los decoders del Transformer, y esto genera que aunque se pueda usar un LLM cuantizado para inferencias, no se pueda entrenar más. Esta consecuencia se da porque las capas o layers que se agregan al LLM no se pueden actualizar.
- Por el punto anterior, es necesario generar adaptadores, los cuales explicamos a continuación.

Según (Voigt, 2025), existen unos componentes llamados `Low-Rank Adapters (LoRA)`, los cuáles normalmente son layers lineales las cuales sí se pueden actualizar. El truco, como lo describe Voigt, es que estas capas son mucho más pequeñas que las layers que se han cuantizado, y como las layers cuantizadas se encuentran congeladas (`frozen`, no se pueden actualizar), al configurar adaptadores es posible reducir el total de parámetros a entrenar a solo  un 1% de los parámetros originales.

Entonces, en resumen, necesitamos de la cuantización para:

1. Cargar el modelo de forma más rápida.
2. Poder congelar el 99% de las capas que lo componen.
3. Insertar adaptadores que permitan entrenar el 1% de capas restantes.


Por último, la cuantización se puede trabajar por medio de las librerías `bitsandbytes` y `peft`.

In [ ]:
# Cargamos el modelo Phi-3 cuantizado en 4 Bits.

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)

# Descargamos el modelo Phi-3 desde Huggingface 
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(
   repo_id, device_map="cuda:0", quantization_config=bnb_config
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# Cargamos el modelo de forma que prepare_model_for_kbit_training permita una mayor estabilidad en el modelo durante el entrenamiento
model = prepare_model_for_kbit_training(model)

# Configuramos nuestros adaptadores
config = LoraConfig(
    # r es el rango del adaptador, y entre mas pequeño sea este, menos parametros entrenaremos.
    r=6,
    
    # Este multiplicador, segun Voigt, debe ser normalmente el doble del valor de r (2*r)
    lora_alpha=12,
    
    # Estas propiedades son recomendadas por Voigt
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    
    # Aqui definimos que layers adaptaremos con LoRa; si se agregan 'gate_up_proj', 'down_proj' el tiempo de entrenamiento aumenta al doble o más
    target_modules = ['o_proj', 'qkv_proj'], # , 'gate_up_proj', 'down_proj'
)

# Cargamos nuestro modelo con la configuracion LoRA
model = get_peft_model(model, config)

In [ ]:
# Con este codigo, es posible saber cuantos parametros estaremos cambiando. En nuestro caso, Phi-3 tiene 3.8 mil millones de parametros, pero solo
# entrenaremos 3.54M para esta entrega

train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')

Trainable parameters:      3.54M
Total parameters:          3824.62M
% of trainable parameters: 0.09%


### Cargando nuestro dataset

Ahora, `Phi-3` tiene la cualidad de que podemos "entrenarlo" como si fuera un Chat. Para esto, existen ciertos tokens o tags que el modelo puede interpretar:

- |user|, que significa que inicia el mensaje o instrucción hacia Phi-3
- |assistant|, que indica el inicio de la respuestra de Phi-3

Para la carga de nuestro dataset de prueba, hicimos uso del estándar conocido como `Alpaca`, que a su vez está basado en el entrenamiento que OpenAI hizo en su modelo `text-davinci-003`. Este estándar incluye 3 distintos tipos de datos por cada elemento:

- `instruction`, la cual es el prompt que deseamos usar como fine-tuning.
- `input`, lo que incluye ciertos datos que pueden estar incluidos o no dentro de la instruction también. En nuestro caso, incluímos estos datos en `instruction` siempre, y algunos autores recomiendan dejar este elemento como vacío para darle liberatad al LLM durante el fine-tuning.
- `output`, que contiene la respuesta esperada por el LLM y que nos sirve para calcular la función de pérdida.


Lo anterior vuelve nuestro entrenamiento como un tipo de NLP Supervisado. Con esto, se considera que se debe hacer el fine-tuning de un modelo con cientos de miles de registros, pero en nuestro caso utilizamos solo 12,500 muestras para acelerar el proceso. Estas muestras se generaron a partir de nuestros `text_features` que generamos para el Avance No. 2, y se puede revisar el notebook `Base_line.ipynb` para confirmar como se generaron los archivos `jsonl` que se utilizaron para este fine-tuning.


***Referencia:***

- R. Tori, et al. (2023). Alpaca: A Strong, Replicable Instruction-Following Model. Recuperado de https://crfm.stanford.edu/2023/03/13/alpaca.html y https://github.com/tatsu-lab/stanford_alpaca

In [ ]:
# Esta funcion genera el formato |user| y |assistant| que Phi-3 puede comprender.
# Se utiliza al cargar los archivos jsonl que generamos en el notebook Base_line.ipynb

def format_dataset(examples):
    # Cargamos nuestro set de instruction-input-
    if isinstance(examples["instruction"], list):
        messages_batch = []
        for i in range(len(examples["instruction"])):
            # Creamos el prompt como instruction e input
            user_message = examples["instruction"][i]
            if examples["input"][i].strip():
                user_message += f"\n\nContexto: {examples['input'][i]}"

            assistant_message = examples["output"][i]

            # Se genera el uso de los Tags |user| y |assistan| para Phi-3
            messages_batch.append([
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ])
        return {"messages": messages_batch}

    else:
        user_message = examples["instruction"]
        if examples["input"].strip():
            user_message += f"\n\nContexto: {examples['input']}"

        assistant_message = examples["output"]

        return {
            "messages": [
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": assistant_message}
            ]
        }

In [ ]:
from datasets import load_dataset
import json
import os

# Usamos la carpeta donde tenemos los archivos jsonl con los sets de instruction-input-output
chunks_dir = "/content/drive/MyDrive/LLM-training/chunks_stream"

# Tomamos solos los archivos jsonl dentro de la carpeta
chunk_files = sorted([os.path.join(chunks_dir, f) for f in os.listdir(chunks_dir) if f.endswith(".jsonl")])

# Indicamos la carpeta dentro de Google Drive donde guardaremos el resultado de nuestro fine-tuning (esto son los adaptadores)
output_dir = "/content/drive/MyDrive/LLM-training/chunk-train-phi3"

# Cargamos el tokenizer de Phi-3 desde HuggingFace
tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# Esta configuracion se obtuve de Voigt (2025), pero cambiamos algunas propiedades para  que la convergencia del modeloa fuera más rápida
sft_config = SFTConfig(
    gradient_checkpointing=False,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=2,
    auto_find_batch_size=True,
    max_length=128,
    packing=False,
    num_train_epochs=1,
    learning_rate=2e-4,
    optim='paged_adamw_8bit',
    logging_steps=50,
    report_to='none',
    output_dir=output_dir,
    bf16=torch.cuda.is_bf16_supported(including_emulation=False),
    max_steps=2500
)

for idx, file_path in enumerate(chunk_files, 1):
    print(f"\n Fine-tuning on chunk {idx}/{len(chunk_files)} → {file_path}")

    # Caragmos el dataset
    dataset = load_dataset("json", data_files=file_path, split="train")
    dataset = dataset.map(format_dataset, remove_columns=["instruction", "input", "output"])

    # Creamos la instancia del trainer para generar los LoRa
    trainer = SFTTrainer(
        model=model.base_model.model,
        peft_config=config,
        processing_class=tokenizer,
        args=sft_config,
        train_dataset=dataset
    )

    # Iniicamos el entrenamiento
    trainer.train()

    # Por cada archivo jsonl que usemos, guardamos el resultado de los LoRA para evitar perder información
    # si generamos algún archivo jsonl incorrectamente
    trainer.save_model(os.path.join(output_dir, f"checkpoint_chunk_{idx:02d}"))



🎯 Fine-tuning on chunk 1/5 → /content/drive/MyDrive/LLM-training/chunks_stream/agave_allThings.jsonl


Map:   0%|          | 0/169011 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/169011 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/169011 [00:00<?, ? examples/s]

...

Tokenizing train dataset:   0%|          | 0/112674 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/112674 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,1.059200
100,0.464700
150,0.445800
200,0.443100
250,0.432000
300,0.432000
350,0.426900
400,0.421500
450,0.433400
500,0.435100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

In [ ]:
# Esta linea permite guardar el resultado final en otra carpeta de Google Drive, como back up.

trainer.save_model("/content/drive/MyDrive/LLM-training/baseline-Phi3-V2")

### Prueba del fine-tuning

`gen_prompt` sirve para aplicar el tokenizer a al prompt que el usuario genere. Como tokenizer usaremos el defualt de Phi-3 para no tener problemas en esta etapa.

In [ ]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [{"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt


`generate` recibe:

- `model`: el resultado del fine-tuning (LLM + adaptadores LoRA)
- `tokenizer`: el tokenizer para convertir los prompts en tokens que Phi-3 puede usar
- `prompt`: lo que el usuario pregunta o escribe
- `max_new_tokens`: la cantidad de tokens que le permitiremos generar a Phi-3.

In [ ]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
  tokenized_input = tokenizer(
  prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  gen_output = model.generate(**tokenized_input,
  eos_token_id=tokenizer.eos_token_id,
  max_new_tokens=max_new_tokens)
  output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)

  return output[0]

#### Prueba del fine-tuning

In [ ]:
sentence = 'Describe un predio con infetacion severa de gorgojos.'
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

<|user|>
Describe un predio con infetacion severa de gorgojos.<|end|>
<|assistant|>



In [ ]:
# Durante nuestra prueba, el resultado obtenido por nuestro entrenamiento (LoRA adapters) fue en 16 bits, pero Phi-3 trabaja en 32 bits
# Entonces debemos convertir nuestros adaptadores a 32 bits para que se entienda con Phi-3

model_2 = model.to(torch.float32)

In [ ]:
# Ponemos a prueba el fine-tuning preguntando por una infestacion moderada de gorgojos

print(generate(model_2, tokenizer, prompt))

<|user|> Describe un predio con infetacion severa de gorgojos.<|end|><|assistant|> La descripción de un predio con infestación severa de gorgojos dependerá de la severidad y extensión del problema. Sin embargo, una descripción general podría ser:

El predio con código 3411 presenta una severidad de riesgo moderada con


# Referencias:

- D. Voigt Godoy. (2025). A Hands-On Guide to Fine-Tuning Large Language Models with PyTorch and Hugging Face. Versión 1.0.1.
- R. Tori, et al. (2023). Alpaca: A Strong, Replicable Instruction-Following Model. Recuperado de https://crfm.stanford.edu/2023/03/13/alpaca.html y https://github.com/tatsu-lab/stanford_alpaca
